## Introdução <a id='introduction'></a>

Métodos de otimização contínua buscam um minimizador local de uma função suave
$$f: \mathbb{R}^n \to \mathbb{R}.$$
Em geral, $f$ é duas vezes continuamente diferenciável (mesmo se as derivadas não estiverem disponíveis ou seu cálculo seja inviável computacionalmente)

Vamos falar sobre o método do Gradiente, ou Método de Cauchy. Assim, dado um ponto inicial $x^0\in \mathbb{R}^n$, procuramos gerar uma sequência de pontos $x^k\in \mathbb{R}^n$ tal que cada nova iteração é dada por 
$$x_{k+1} = x_k + \alpha_k p_k$$
em que $p_k$ é uma *direção de descida*, ou seja, é uma direção na qual sabemos que a função objetivo tem seu valor reduzido (ao menos localmente). Então, $p_k$ deve satisfazer a condição
$$p_k^T\nabla f(x_k) < 0.$$
Em geral, $\| p_k\|_2=1$, e assim $\alpha_k$ determina o tamanho do passo que será tomado nessa direção. 

Repetiremos essa iteração até que seja encontrado um ponto para o qual $\nabla f(x_k)=0$ ou $|\nabla f(x_k)|<\epsilon$, em que $\epsilon >0$ é alguma tolerância (em geral, algo em torno de $10^{-10}$, por exemplo).

### Função de teste

A <a href="https://en.wikipedia.org/wiki/Rosenbrock_function">função de Rosenbrock</a> é uma função conhecida para se testar algoritmos de otimização irrestrita. A forma usual é 
$$f(x) = (1-x_0)^2 + \beta(x_1-x_0^2)^2$$
em que $\beta$ é algum número real (normalmente positivo). Vamos usar $\beta=10$:

In [ ]:
def rosenbrock(x):
    return (1-x[0])**2+10*(x[1]-x[0]**2)**2

Agora, podemos calcular o valor da função de Rosenbrock em um determinado ponto:

In [ ]:
x = [2,2]

ao fazermos

In [ ]:
rosenbrock(x)

### Método do gradiente

Conforme já discutido em aula, vamos usar aqui $p_k = -\nabla f(x_k)$ em todas as iterações. Para isso, vamos definir uma função que calcula o gradiente da função:

In [ ]:
import numpy as np

In [ ]:
def gradiente(x):
    return np.asarray([2*x[0]-2-40*x[0]*(x[1]-x[0]**2), 20*(x[1]-x[0]**2)])

(observe que definimos a saída da função gradiente como uma ndarray, pois isso será necessário mais à frente)

Então, escolha um ponto inicial:

In [ ]:
x = [0,0]

A direção aqui será

In [ ]:
p = -gradiente(x)

Nessa direção, se tomarmos um passo com $\alpha^k=1$, teremos:

In [ ]:
ponto_inicial = np.asarray(x)
ponto_novo = ponto_inicial + p

Assim

In [ ]:
ponto_novo

Vamos ver na figura:

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
def grafico_curvas(points):
    x = np.linspace(-2, 2, 100)
    y = np.linspace(-2, 2, 100)
    X, Y = np.meshgrid(x, y)
    R = rosenbrock([X, Y])
    plt.figure()
    levels = [0, 0.1, 0.5, 1, 2, 5, 10, 20, 30, 50, 100]
    plt.contour(X, Y, R, levels)

    for point in points:
        plt.plot(point[0], point[1], 'ro')

In [ ]:
grafico_curvas((ponto_inicial, ponto_novo))
plt.annotate("x_0", ponto_inicial, xytext=ponto_inicial-np.asarray([0.3, 0]))
plt.annotate("x_1", ponto_novo, xytext=ponto_novo-np.asarray([0.2, 0.3]))

No entanto, um passo nessa direção na verdade *aumentou* o valor da função:

In [ ]:
rosenbrock(ponto_inicial)

In [ ]:
rosenbrock(ponto_novo)

## Backtracking

Já vimos em aula que as condições de Wolfe garantem que poderemos encontrar um $alpha_k$ apropriado que (i) resulta em um *decréscimo suficiente* no valor da função objetivo; e (ii) satisfaz a condição de curvatura. No entanto, encontrar esse ponto diretamente pode ser difícil. 

Propomos então a estratégia seguinte: vamos escolher o maior valor de $\alpha$ tal que $x_k+\alpha p_k$ satisfaça a condição de decréscimo suficiente. (vamos criar uma função para esse procedimento para podermos utilizá-la novamente mais tarde):

In [ ]:
def buscalinear(ponto_inicial, p, c1=1e-4):
    alpha = 1 # A escolha do alpha inicial também pode ser discutida, 
              # mas aqui escolheremos simplesmente alpha=1.
    f_inicial = rosenbrock(ponto_inicial)
    grad_inicial = gradiente(ponto_inicial)
    ponto_novo = ponto_inicial + alpha*p
    while rosenbrock(ponto_novo) > f_inicial + c1*alpha*np.dot(grad_inicial, p):
        alpha = 0.9*alpha
        ponto_novo = ponto_inicial + alpha*p
    return ponto_novo

In [ ]:
ponto_novo = buscalinear(ponto_inicial, p)

In [ ]:
ponto_novo

Na figura:

In [ ]:
grafico_curvas((ponto_inicial, ponto_novo))

Agora sim:

In [ ]:
rosenbrock(ponto_novo)

Vamos repetir o processo até que estejamos satisfeitos com o valor do gradiente da função. 

In [ ]:
def metodo_do_gradiente(fun, grad, x, tolerancia_gradiente):
    p = -grad(x)
    while np.sqrt(p[0]**2+p[1]**2) > tolerancia_gradiente:
        x = buscalinear(x, p)
        p = -grad(x)
    return x

In [ ]:
solucao = metodo_do_gradiente(rosenbrock, gradiente, ponto_inicial, 1e-3)

De fato,

In [ ]:
gradiente(solucao)

Finalmente, podemos modificar nosso método do gradiente para guardarmos todas as iteradas e observarmos o caminho do método do gradiente:

In [ ]:
def metodo_do_gradiente(fun, grad, x, tolerancia_gradiente):
    iteradas = []
    # iteradas é uma lista que guarda todos os pontos gerados pelo método
    iteradas.append(x)
    p = -grad(x)
    while np.sqrt(p[0]**2+p[1]**2) > tolerancia_gradiente:
        x = buscalinear(x, p)
        iteradas.append(x)
        p = -grad(x)
    return x, iteradas

In [ ]:
solucao, iteradas = metodo_do_gradiente(rosenbrock, gradiente, ponto_inicial, 1e-3)

In [ ]:
iteradas

In [ ]:
grafico_curvas(iteradas)

Ok! Achamos uma solução. Mas o método não é muito bom: levamos

In [ ]:
len(iteradas)

para encontrarmos a solução...